In [1]:
using Flux

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/edoardo/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


In [2]:
versioninfo()

Julia Version 1.3.0
Commit 46ce4d7933 (2019-11-26 06:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


In [3]:
mutable struct MinimalRNNCell{T,H}
    kernel::T
    recurrent_kernel::T
    hidden::H
end

function MinimalRNNCell(input_shape, units::Integer)
    kernel = rand(input_shape[end], units)*0.1.-0.05
    recurrent_kernel = rand(units, units)*0.1.-0.05
    MinimalRNNCell(kernel, recurrent_kernel, zeros(units))
end

function (c::MinimalRNNCell)(hᵢ₋₁::AbstractArray, x::AbstractArray)
    output = (c.kernel * x) + (c.recurrent_kernel * hᵢ₋₁)
    c.hidden = output
    return output, output
end
Flux.hidden(c::MinimalRNNCell) = c.hidden

Flux.@functor MinimalRNNCell

In [4]:
function GenerateSample(length, output)
    inputs = randn(length, output) * 0.1
    outputs = zeros((length,output))
    for i in 1:length
        if i == 1
            outputs[i,:] = inputs[i,:]
        else
            outputs[i,:] = inputs[i,:] + inputs[i-1,:] 
        end
    end
    inputs, outputs
end

GenerateSample (generic function with 1 method)

In [5]:
train_x = zeros(50,10,3)
train_y = zeros(50,10,3)
for i in 1:50
   train_x[i,:,:], train_y[i,:,:] = GenerateSample(10,3) 
end

In [14]:
opt = Flux.ADAM()
function loss(model, x,y)
    total_loss = 0.
    for i in 1:size(x,1)
        for j in 1:size(x,2)
            @inbounds total_loss += Flux.mse(model(x[i,j,:]),y[i,j,:])
        end    
        Flux.reset!(model)
    end
    total_loss / size(x,1)
end
m = Chain(Flux.Recur(MinimalRNNCell((10,3),3)))

Chain(Recur(MinimalRNNCell{Array{Float64,2},Array{Float64,1}}([0.021965766141725365 -0.04484158145864097 0.03765786082256732; -0.013919994877346277 0.040177198269033126 -0.012428416982709356; 0.01777357593503484 -0.007188381274188929 -0.015239923989250802], [-0.0461493766732751 0.02375341354285472 0.03705982736386296; -0.022759201890946268 -0.0036592526757153396 0.03613255917080056; -0.019441542085982255 -0.004312042151815174 -0.04169944625267066], [0.0, 0.0, 0.0])))

In [7]:
# Set up hidden state
m(train_x[1,1,:])
@code_warntype m.layers[1].cell(m.layers[1].state, train_x[1,1,:])

Variables
  c::MinimalRNNCell{Array{Float64,2},Array{Float64,1}}
  hᵢ₋₁::Array{Float64,1}
  x::Array{Float64,1}
  output::Array{Float64,1}

Body::Tuple{Array{Float64,1},Array{Float64,1}}
1 ─ %1 = Base.getproperty(c, :kernel)::Array{Float64,2}
│   %2 = (%1 * x)::Array{Float64,1}
│   %3 = Base.getproperty(c, :recurrent_kernel)::Array{Float64,2}
│   %4 = (%3 * hᵢ₋₁)::Array{Float64,1}
│        (output = %2 + %4)
│        Base.setproperty!(c, :hidden, output)
│   %7 = Core.tuple(output, output)::Tuple{Array{Float64,1},Array{Float64,1}}
└──      return %7


In [15]:
function fit!(model, train_x, train_y, epochs; opt=Flux.ADAM(), ps=params(model) )
    Flux.reset!(model)
    for i in 1:epochs
        println("Epoch $i")
         start = time_ns()
        gs = gradient(ps) do
            loss(model, train_x, train_y)
        end
        Flux.Optimise.update!(opt, ps, gs)
        stop = time_ns()
        println("$((stop-start) / 1000 ) μs")
    end
end

fit! (generic function with 1 method)

In [16]:
fit!(m, train_x, train_y, 10)

Epoch 1
259139.776 μs
Epoch 2
82188.526 μs
Epoch 3
64451.455 μs
Epoch 4
77448.534 μs
Epoch 5
58878.625 μs
Epoch 6
74176.883 μs
Epoch 7
56729.679 μs
Epoch 8
75344.094 μs
Epoch 9
59201.865 μs
Epoch 10
73786.638 μs


In [22]:
using Profile
ps = params(m)
gs = gradient(ps) do
    total_loss = 0.
    for i in 1:size(train_x,1)
        for j in 1:size(train_x,2)
            @inbounds total_loss += Flux.mse(m(train_x[i,j,:]),train_y[i,j,:])
        end    
        Flux.reset!(m)
    end
    total_loss
end
gs.grads

IdDict{Any,Any} with 1006 entries:
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => Ref

In [20]:
@profile gradient(ps) do
    total_loss = 0.
    for i in 1:size(train_x,1)
        for j in 1:size(train_x,2)
            @inbounds total_loss += Flux.mse(m(train_x[i,j,:]),train_y[i,j,:])
        end    
        Flux.reset!(m)
    end
    total_loss
end
Profile.print()

Grads(...)

IdDict{Any,Any} with 1006 entries:
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => RefValue{Any}((x = nothing,))
  RefValue{Val{2}}(Val{2}()) => RefValue{Any}((x = nothing,))
  RefValue{typeof(^)}(^)     => Ref